In [1]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import sys
from pytorch_lightning.loggers import TensorBoardLogger

# Set the root directory to Sorrento Home path
root_path = Path.cwd().parents[0]
sys.path.append(str(root_path))


In [58]:
%load_ext tensorboard
%tensorboard --logdir ../lightning_logs/

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6008 (pid 230773), started 0:00:05 ago. (Use '!kill 230773' to kill it.)

In [36]:
from tbparse import SummaryReader
log_dir = "../lightning_logs/JTK_Challenge/version_0/"
reader = SummaryReader(log_dir, pivot=True)
df = reader.scalars
print(df)

      step           epoch  hp_metric  train_acc  train_fbeta  train_loss  \
0        0             NaN       -1.0        NaN          NaN         NaN   
1       49             0.0        NaN    0.65625     0.378788    0.652252   
2       99             0.0        NaN    0.68750     0.350877    0.494739   
3      105             0.0        NaN        NaN          NaN         NaN   
4      149             1.0        NaN    0.81250     0.795455    0.291985   
..     ...             ...        ...        ...          ...         ...   
766  26349           248.0        NaN    1.00000     1.000000    0.000589   
767  26393           248.0        NaN        NaN          NaN         NaN   
768  26399           249.0        NaN    1.00000     1.000000    0.001944   
769  26449           249.0        NaN    1.00000     1.000000    0.022969   
770  26499  [249.0, 249.0]        NaN    1.00000     1.000000    0.000561   

      val_acc  val_fbeta  val_loss  
0         NaN        NaN       NaN  
1

In [43]:
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
import pandas as pd

# Path to your TensorBoard log files
#log_dir = 'path_to_your_logs'
log_dir = "../lightning_logs/JTK_Challenge/"

# Initialize an event accumulator
event_acc = EventAccumulator(log_dir,
                             size_guidance={  # see the tensorboard docs for more details on these parameters
                                 'scalars': 0,  # 0 means load all scalar events
                             })

# Load all events from the log files
event_acc.Reload()

# Assuming you have scalar data and you know the tag names, you can convert them to a pandas DataFrame
# Replace 'scalar_tag' with your actual tag name
scalar_tag = 'val_fbeta'  # example: 'loss' or 'accuracy'
scalar_events = event_acc.Scalars(scalar_tag)

# Convert to DataFrame
df = pd.DataFrame([(e.wall_time, e.step, e.value) for e in scalar_events],
                  columns=['wall_time', 'step', 'value'])

# Optionally, convert wall_time (UNIX timestamp) to a human-readable format
df['wall_time'] = pd.to_datetime(df['wall_time'], unit='s')


KeyError: 'Key val_fbeta was not found in Reservoir'

In [55]:
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
import pandas as pd
import os

# Main directory containing all runs
main_log_dir = "../lightning_logs/JTK_Challenge/"

# Initialize an empty DataFrame to hold all data
all_runs_df = pd.DataFrame()

# Iterate over each subdirectory (each run)
for i, run_dir in enumerate(os.listdir(main_log_dir)):
    run_path = os.path.join(main_log_dir, run_dir)
    if os.path.isdir(run_path):  # Check if it's a directory
        # Initialize an event accumulator for the current run
        event_acc = EventAccumulator(run_path, size_guidance={'scalars': 0})
        event_acc.Reload()

        # Assuming you are interested in scalar data
        scalar_tag = 'val_fbeta'  # Replace with your actual tag
        if scalar_tag in event_acc.Tags()['scalars']:  # Check if the scalar tag exists
            scalar_events = event_acc.Scalars(scalar_tag)

            # Convert to DataFrame for the current run
            run_df = pd.DataFrame([(e.wall_time, e.step, e.value) for e in scalar_events],
                                  columns=['wall_time', 'step', 'value'])

            # Add run identifier: here we use run_dir as identifier, but you could also use i (the run number)
            run_df['run_id'] = run_dir  # or use `i` for a numerical identifier

            # Optionally, convert wall_time to a readable format
            run_df['wall_time'] = pd.to_datetime(run_df['wall_time'], unit='s')

            # Append the current run's data to the all_runs_df DataFrame
            all_runs_df = pd.concat([all_runs_df, run_df], ignore_index=True)

In [56]:
print(all_runs_df.head(250))

                        wall_time   step     value      run_id
0   2024-03-30 02:17:29.113399744     52  0.034853  version_91
1   2024-03-30 02:17:29.885790825    105  0.797964  version_91
2   2024-03-30 02:17:30.362987757    158  0.122309  version_91
3   2024-03-30 02:17:30.881132364    211  0.045010  version_91
4   2024-03-30 02:17:31.354292631    264  0.868784  version_91
..                            ...    ...       ...         ...
245 2024-03-30 02:19:31.351519346  13037  1.000000  version_91
246 2024-03-30 02:19:31.864068747  13090  0.992860  version_91
247 2024-03-30 02:19:32.359914541  13143  0.992860  version_91
248 2024-03-30 02:19:32.878743887  13196  0.992860  version_91
249 2024-03-30 02:19:33.377018690  13249  0.992860  version_91

[250 rows x 4 columns]


In [30]:
#reader.scalars.pivot(index="epoch", columns="tag", values="value")
output = {}
for tag in reader.scalars["tag"].unique():
    output[tag] = reader.scalars[reader.scalars["tag"] == tag]["value"].values
    

In [35]:
reader.scalars["tag"].value_counts()

tag
epoch          780
train_acc      530
train_fbeta    530
train_loss     530
val_acc        250
val_fbeta      250
val_loss       250
hp_metric        1
Name: count, dtype: int64

In [34]:
pd.DataFrame(output)

ValueError: All arrays must be of the same length

In [31]:
import pandas as pd